In [1]:
%gui qt
import os.path as op
import mne

sample_path = mne.datasets.sample.data_path()
data_path = op.join(sample_path, 'MEG', 'sample')
subjects_dir = op.join(sample_path, 'subjects')

In [2]:
inv = mne.minimum_norm.read_inverse_operator(op.join(data_path, 'sample_audvis-meg-oct-6-meg-fixed-inv.fif'))

In [3]:
raw = mne.io.read_raw_fif(op.join(data_path, 'sample_audvis_filt-0-40_raw.fif'))
events = mne.read_events(op.join(data_path, 'sample_audvis_filt-0-40_raw-eve.fif'))
event_id = {'audio/left': 1,
            'audio/right': 2,
            'visual/left': 3,
            'visual/right': 4}
epochs = mne.Epochs(raw, events, event_id)

In [4]:
src = mne.read_source_spaces(op.join(subjects_dir, 'sample', 'bem', 'sample-all-src.fif'))

In [5]:
epochs_stc = mne.minimum_norm.apply_inverse_epochs(epochs, inv, lambda2=0.1111)

In [6]:
import rsa
import numpy as np

In [8]:
model_dsm = np.array([
    [0, 0.5, 1, 1],
    [0.5, 0, 1, 1],
    [1, 1, 0, 0.5],
    [1, 1, 0.5, 0],
])
rsa_vals = rsa.rsa_source_level(epochs_stc, model_dsm, inv['src'], y=epochs.events[:, 2], n_folds=5, n_jobs=2, verbose=True)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


PicklingError: Could not pickle the task to send it to the workers.

  2%|▏         | 14032/577346 [00:30<07:26, 1260.59it/s]

In [ ]:
rsa_vals.plot('sample', hemi='both', subjects_dir=subjects_dir)

In [ ]:
inv.keys()

In [ ]:
inv['src']

In [ ]:
epochs_stc[0]

In [ ]:
len(inv['src'][0]['vertno']) + len(inv['src'][1]['vertno'])

In [ ]:
np.sum(inv['src'][0]['inuse']) + np.sum(inv['src'][1]['inuse'])

In [ ]:
%debug

> /work/modules/Ubuntu/14.04/amd64/common/anaconda3/latest/envs/neuroimaging/lib/python3.6/concurrent/futures/_base.py(384)__get_result()
    382     def __get_result(self):
    383         if self._exception:
--> 384             raise self._exception
    385         else:
    386             return self._result

ipdb> upo
*** NameError: name 'upo' is not defined
ipdb> up
> /work/modules/Ubuntu/14.04/amd64/common/anaconda3/latest/envs/neuroimaging/lib/python3.6/concurrent/futures/_base.py(432)result()
    430                 raise CancelledError()
    431             elif self._state == FINISHED:
--> 432                 return self.__get_result()
    433             else:
    434                 raise TimeoutError()

ipdb> up
> /m/work/modules/Ubuntu/14.04/amd64/common/anaconda3/latest/envs/neuroimaging/lib/python3.6/site-packages/joblib/_parallel_backends.py(521)wrap_future_result()
    519         AsyncResults.get from multiprocessing."""
    520         try:
--> 521             retu